### Importing the libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import os
import subprocess

### The base path of the folder where the data is stored.

In [4]:
path='C:\\Users\\asd\\Desktop\\sort-tracker\\India 2\\500 frames'

### Example DataFrame

In [5]:
readFile=pd.read_csv(path+"/1/2/002.csv")
print(readFile)

      frameNumber        label  id  x_TL  y_TL  x_BR  y_BR  isOccluded  \
0               0          Car   0   332   255   523   160           0   
1               0          Car   1    40   186    75   141           0   
2               0  Two-Wheeler   2   612   233   663   198           0   
3               0   Pedestrian   3   810   207   823   178           0   
4               0   Pedestrian   4   800   207   811   175           0   
5               1          Car   0   315   255   510   158           0   
6               1          Car   1    31   186    66   141           0   
7               1  Two-Wheeler   2   603   233   656   196           0   
8               1   Pedestrian   3   804   207   817   177           0   
9               1   Pedestrian   4   795   207   806   174           0   
10              2          Car   0   298   256   498   156           0   
11              2          Car   1    23   186    58   141           0   
12              2  Two-Wheeler   2   5

### A Survey of ID list and Label list

In [6]:
labels=readFile.label
ids=readFile.id
lines=len(readFile)
idList=[]
labelList=[]
for i in range(lines):
    if ids[i] in idList:
        continue
    else:
        idList.append(ids[i])
        labelList.append(labels[i])
print(idList)
print(len(idList))
print(labelList)

[0, 1, 2, 3, 4, 6, 5, 7, 8, 9, 12, 11, 10, 13, 14, 16, 15]
17
['Car', 'Car', 'Two-Wheeler', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Two-Wheeler', 'Car', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Pedestrian', 'Car', 'Car', 'Car']


# Benchmarking

`totalFrames` stores the total number of the frames that we have for processing

`totalID` Stores the total number of distinct objects we have for processing

`videoNo` Stores the total number of annotated sequences or videos we have

`totalbbox` gives the total number of bounding boxes we have in the entire dataset

`totalPeds` gives the total number of annotated bounding boxes occluded as well as unoccluded

`occPeds` gives the total number of occluded annotated pedestrian bounding boxes

Similar data is plotted for Bus, Two wheelers, cars, Trucks, Autos, Cyclists and Animals respectively

`irrcount` represents the number of bounding boxes of annotated road signs

`ijkPedCross` `ijkStop` are lists which stores the video folder of annotators, the corresponding video and frames in which the road signs data has been annotated.



In [7]:
def height(ybr,ytl):
    return abs(ybr-ytl)
def width(xbr,xtl):
    return abs(xbr-xtl)
def avg(arr,total):
    npArr=np.asarray(arr)
    average=np.sum(npArr)/total
    return average

In [8]:
totalFrames=0
totalID=0
videoNo=0
totalbbox=0
totalPeds=0
occPeds=0
totalBus=0
occBus=0
totalTwo=0
occTwo=0
totalCar=0
occCar=0
totalTruck=0
occTruck=0
totalAuto=0
occAuto=0
totalCyclist=0
occCyclist=0
totalAnimal=0
occAnimal=0
allClasses=['Car', 'Two-Wheeler', 'Pedestrian', 'Auto-Rickshaw', 'Truck', 'Cyclist', 'Bus', 'Animal', 'Pedestrian Crossing', 'No Parking', 'No Stopping', 'Right Turn', 'U-Turn']
irrcount=0
ijkPedCross=[]
ijkPark=[]
ijkStop=[]
ijkRight=[]
ijkU=[]
pedWidth=[]
pedHeight=[]
carWidth=[]
carHeight=[]
busWidth=[]
busHeight=[]
autoWidth=[]
autoHeight=[]
truckWidth=[]
truckHeight=[]
twoWidth=[]
twoHeight=[]
cyclistWidth=[]
cyclistHeight=[]
animalWidth=[]
animalHeight=[]
uniqPeds=0
uniqCars=0
uniqTrucks=0
uniqBuses=0
uniqCyclists=0
uniqTwos=0
uniqAnimals=0
uniqAutos=0
#i represents the outer folders representing different persons' video folders
for i in range(1,20):
    inpath=path+"/"+str(i)
    for j in range(1,8):
        try:
            fullPath=inpath+"/"+str(j)+"/00"+str(j)+".csv"#j loop represents the name of the video folder and the video we are trying to extract
            readFile=pd.read_csv(fullPath)
            totalbbox=totalbbox+len(readFile)
            pedsIDuniq=[]
            carsIDuniq=[]
            busesIDuniq=[]
            trucksIDuniq=[]
            cyclistsIDuniq=[]
            twosIDuniq=[]
            animalsIDuniq=[]
            autosIDuniq=[]
            totalFrames=totalFrames + max(readFile.frameNumber)#total frames added
            ids=readFile.id
            idSet=set(ids)
            totalID=totalID+len(idSet)#total number of dstinct ids or total number of objects present in all the videos
            videoNo=videoNo+1#total number of videos
            for k in range(len(readFile)):
                if(readFile.label[k]=='Pedestrian'):
                    totalPeds=totalPeds+1
                    pedWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    pedHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in pedsIDuniq)):
                        uniqPeds=uniqPeds+1
                        pedsIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occPeds=occPeds+1
                elif(readFile.label[k]=='Car'):
                    totalCar=totalCar+1
                    carWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    carHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in carsIDuniq)):
                        uniqCars=uniqCars+1
                        carsIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occCar=occCar+1
                elif(readFile.label[k]=='Truck'):
                    totalTruck=totalTruck+1
                    truckWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    truckHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in trucksIDuniq)):
                        uniqTrucks=uniqTrucks+1
                        trucksIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occTruck=occTruck+1
                elif(readFile.label[k]=='Bus'):
                    totalBus=totalBus+1
                    busWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    busHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in busesIDuniq)):
                        uniqBuses=uniqBuses+1
                        busesIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occBus=occBus+1
                elif(readFile.label[k]=='Two-Wheeler'):
                    totalTwo=totalTwo+1
                    twoWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    twoHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in twosIDuniq)):
                        uniqTwos=uniqTwos+1
                        twosIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occTwo=occTwo+1
                elif(readFile.label[k]=='Auto-Rickshaw'):
                    totalAuto=totalAuto+1
                    autoWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    autoHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in autosIDuniq)):
                        uniqAutos=uniqAutos+1
                        autosIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occAuto=occAuto+1
                elif(readFile.label[k]=='Cyclist'):
                    totalCyclist=totalCyclist+1
                    cyclistWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    cyclistHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in cyclistsIDuniq)):
                        uniqCyclists=uniqCyclists+1
                        cyclistsIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occCyclist=occCyclist+1
                elif(readFile.label[k]=='Animal'):
                    totalAnimal=totalAnimal+1
                    animalWidth.append(width(readFile.x_BR[k],readFile.x_TL[k]))
                    animalHeight.append(height(readFile.y_BR[k],readFile.y_TL[k]))
                    if(not(readFile.id[k] in animalsIDuniq)):
                        uniqAnimals=uniqAnimals+1
                        animalsIDuniq.append(readFile.id[k])
                    if(readFile.isOccluded[k]==1):
                        occAnimal=occAnimal+1
                elif(readFile.label[k]=='Pedestrian Crossing'):
                    irrcount=irrcount+1
                    ijkPedCross.append(str(i)+"-"+str(j)+"-"+str(k))
                elif(readFile.label[k]=='No Parking'):
                    irrcount=irrcount+1
                    ijkPark.append(str(i)+"-"+str(j)+"-"+str(k))
                elif(readFile.label[k]=='No Stopping'):
                    irrcount=irrcount+1
                    ijkStop.append(str(i)+"-"+str(j)+"-"+str(k))
                elif(readFile.label[k]=='Right Turn'):
                    irrcount=irrcount+1
                    ijkRight.append(str(i)+"-"+str(j)+"-"+str(k))
                elif(readFile.label[k]=='U-Turn'):
                    irrcount=irrcount+1
                    ijkU.append(str(i)+"-"+str(j)+"-"+str(k))
                elif(not(readFile.label[k] in allClasses)):
                    allClasses.append(readFile.label[k])
        except:
            print("file not found",i,j)
            videoNo=videoNo-1#unfound videos are unadded from the total video number


file not found 1 1
file not found 3 7
file not found 4 4
file not found 4 5
file not found 4 6
file not found 14 5
file not found 14 6
file not found 14 7
file not found 15 3
file not found 15 4
file not found 15 5
file not found 15 6
file not found 15 7
file not found 16 5
file not found 16 6
file not found 16 7
file not found 17 5
file not found 17 6
file not found 17 7
file not found 18 5
file not found 18 6
file not found 18 7
file not found 19 5
file not found 19 6
file not found 19 7


In [9]:
print("Total number of frames",totalFrames)
print("Total number of distinct ids or total number of distinct objects",totalID)
print("Total sequences or videos",videoNo)
print("Total Number of Bounding Boxes", totalbbox)
print("Total Number of pedestrian bounding boxes", totalPeds )
print("occluded pedestrian bounding boxes",occPeds)
print("number of unique pedestrians",uniqPeds)
print("Total Number of car bounding boxes", totalCar )
print("occluded cars",occCar)
print("number of unique Cars",uniqCars)
print("Total Number of bus bounding boxes", totalBus )
print("occluded buses",occBus)
print("number of unique buses",uniqBuses)
print("Total Number of Truck bounding boxes", totalTruck )
print("occluded Trucks",occTruck)
print("number of unique trucks",uniqTrucks)
print("Total Number of Cyclist bounding boxes", totalCyclist )
print("occluded Cyclists",occCyclist)
print("number of unique cyclists",uniqCyclists)
print("Total Number of two-wheeler bounding boxes", totalTwo )
print("occluded two-wheelers",occTwo)
print("number of unique two-wheelers",uniqTwos)
print("Total Number of Auto-Rikshaw bounding boxes", totalAuto )
print("occluded Auto-Rikshaws",occAuto)
print("number of unique autos",uniqAutos)
print("Total Number of Animal bounding boxes", totalAnimal )
print("occluded Animals",occAnimal)
print("number of unique animals",uniqAnimals)
print("totalbbox", totalPeds+totalCar+totalBus+totalTruck+totalCyclist+totalTwo+totalAuto+totalAnimal)
print("Total number of bounding boxes for the 8 class objects", totalbbox-irrcount)
print("All the classes of data that are present",allClasses[0:3])
print(allClasses[3:8])
print(allClasses[8:])
avgPedWidth=avg(pedWidth,totalPeds)
avgPedHeight=avg(pedHeight,totalPeds)
avgCarWidth=avg(carWidth,totalCar)
avgCarHeight=avg(carHeight,totalCar)
avgTruckWidth=avg(truckWidth,totalTruck)
avgTruckHeight=avg(truckHeight,totalTruck)
avgBusWidth=avg(busWidth,totalBus)
avgBusHeight=avg(busHeight,totalBus)
avgAnimalWidth=avg(animalWidth,totalAnimal)
avgAnimalHeight=avg(animalHeight,totalAnimal)
avgTwoWidth=avg(twoWidth,totalTwo)
avgTwoHeight=avg(twoHeight,totalTwo)
avgCyclistWidth=avg(pedWidth,totalCyclist)
avgCyclistHeight=avg(pedHeight,totalCyclist)
avgAutoWidth=avg(pedWidth,totalAuto)
avgAutoHeight=avg(pedHeight,totalAuto)
print("average pedestrian width",avgPedWidth)
print("average pedestrian Height",avgPedHeight)
print("average Car width",avgCarWidth)
print("average Car Height",avgCarHeight)
print("average Bus width",avgBusWidth)
print("average Bus Height",avgBusHeight)
print("average Auto width",avgAutoWidth)
print("average Auto Height",avgAutoHeight)
print("average Cyclist width",avgCyclistWidth)
print("average Cyclist Height",avgCyclistHeight)
print("average Two Wheeler width",avgTwoWidth)
print("average Two Wheeler Height",avgTwoHeight)
print("average Truck width",avgTruckWidth)
print("average Truck Height",avgTruckHeight)
print("average Animal width",avgAnimalWidth)
print("average Animal Height",avgAnimalHeight)
print("Number of road sign boxes",irrcount)
print("The person-video-frame info for annotation of Pedestrian Crossing marking \n",ijkPedCross)
print("The person-video-frame info for annotation of Parking Sign marking \n",ijkPark)
print("The person-video-frame info for annotation of Stop Sign marking \n",ijkStop)
print("The person-video-frame info for annotation of Right Turn marking \n",ijkRight)
print("The person-video-frame info for annotation of U-Turn marking \n",ijkU)


Total number of frames 53797
Total number of distinct ids or total number of distinct objects 2493
Total sequences or videos 83
Total Number of Bounding Boxes 253468
Total Number of pedestrian bounding boxes 20036
occluded pedestrian bounding boxes 2515
number of unique pedestrians 371
Total Number of car bounding boxes 143051
occluded cars 32944
number of unique Cars 1325
Total Number of bus bounding boxes 7244
occluded buses 2017
number of unique buses 44
Total Number of Truck bounding boxes 10462
occluded Trucks 2511
number of unique trucks 79
Total Number of Cyclist bounding boxes 3956
occluded Cyclists 743
number of unique cyclists 58
Total Number of two-wheeler bounding boxes 50907
occluded two-wheelers 5981
number of unique two-wheelers 467
Total Number of Auto-Rikshaw bounding boxes 17436
occluded Auto-Rikshaws 2699
number of unique autos 146
Total Number of Animal bounding boxes 131
occluded Animals 0
number of unique animals 4
totalbbox 253223
Total number of bounding boxes f